In [1]:
import tensorflow as tf
import tensorflow.contrib.layers as layers
config = tf.ConfigProto(intra_op_parallelism_threads=8, inter_op_parallelism_threads=8, \
                        allow_soft_placement=True, device_count = {'CPU': 8})
sess = tf.InteractiveSession(config = config)

In [2]:
import numpy as np

In [3]:
def EM_network(data_P, data_Q, name):
    with tf.variable_scope(name):  
        data_combined = tf.concat([data_P, data_Q], axis = 0)
        
        lay = layers.relu(data_combined, 20)
        lay = layers.relu(lay, 20)
        outputs = layers.linear(lay, 1)
        
    these_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope = name)
    
    return outputs, these_vars

In [4]:
def EM_loss(KL_output, name):
    with tf.variable_scope(name):
        batch_size_dyn = tf.cast(tf.math.divide(tf.shape(KL_output)[0], 2), tf.int32)
        
        T_P = KL_output[:batch_size_dyn,:]
        T_Q = KL_output[batch_size_dyn:,:]
        
        EM_loss = tf.reduce_mean(T_P, axis = 0) - tf.reduce_mean(T_Q, axis = 0)
        
        EM_loss = EM_loss[0]
        
    return EM_loss

In [5]:
data_P_in = tf.placeholder(tf.float32, [None, 1], name = 'data_P_in')
data_Q_in = tf.placeholder(tf.float32, [None, 1], name = 'data_Q_in')

In [6]:
EM_output, EM_vars = EM_network(data_P_in, data_Q_in, name = 'EM')

In [7]:
EM_lossval = EM_loss(EM_output, 'EM_loss')

In [8]:
train_EM = tf.train.AdamOptimizer(learning_rate = 0.005, beta1 = 0.3, beta2 = 0.5).minimize(-EM_lossval, var_list = EM_vars)

In [36]:
clipval = 0.2

In [70]:
clip_EM_network = [var.assign(tf.clip_by_value(var, -clipval, clipval)) for var in EM_vars]

In [80]:
sess.run(tf.global_variables_initializer())

In [81]:
data_Q_train = np.loadtxt("cls_bkg.txt")
data_P_train = np.loadtxt("target_bkg.txt")

In [82]:
data_Q_train = np.expand_dims(data_Q_train, axis = 1)
data_P_train = np.expand_dims(data_P_train, axis = 1)

In [83]:
for epoch in range(900):
    sess.run([train_EM, clip_EM_network], feed_dict = {data_P_in: data_P_train, data_Q_in: data_Q_train})
    loss = sess.run(EM_lossval, feed_dict = {data_P_in: data_P_train, data_Q_in: data_Q_train})
    
    if not epoch % 200:
        print("epoch {}: loss = {}".format(epoch, loss))

epoch 0: loss = -9.894371032714844e-06
epoch 200: loss = 0.0002510547637939453
epoch 400: loss = 0.0035764575004577637
epoch 600: loss = 0.006959319114685059
epoch 800: loss = 0.0064705610275268555
